In [2]:
import sys
import os
import numpy as np
import multiprocessing
import dill
import matplotlib.pyplot as plt
import pandas as pd
import sklearn 
from sklearn.preprocessing import PolynomialFeatures

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/../src/simulations_v2")
    sys.path.append(module_path + "/..")
from load_params import load_params

from multi_group_simulation import MultiGroupSimulation

from util_functions import *
from uncertainty_analysis import *
from sim_helper_functions import *
from plot_utils import *
configure_plot(plt)

lhs_output_sim_files = []
for i in range(2000):
    
    # replace fname with local path!!!
    #fname = '/Users/brianliu/Summer_2021_Research/CovidDelta/COVID-PNAS/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    fname = '/home/yz685/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    lhs_output_sim_files.append(fname)

In [3]:
def residential_regression_non_linear(scenario_data):
    residential_columns = scenario_data.columns[0:12]
    residential_target = 'res_cornell_inf_50'
    X_res = scenario_data[residential_columns]
    Y_res_outcomes = np.array(scenario_data[[residential_target]])

    X_res = scenario_data[residential_columns]
    quadratic = PolynomialFeatures(degree = 2,interaction_only=False,include_bias = False)
    X_quadratic = pd.DataFrame(quadratic.fit_transform(X_res),columns = quadratic.get_feature_names(X_res.columns))
    X = add_constant(X_quadratic)
    model = OLS(Y_res_outcomes,X)
    results = model.fit()
    
    return model

In [4]:
scenario_data = load_sim_output(lhs_output_sim_files)
reg_model = residential_regression_non_linear(scenario_data)
reg_results = reg_model.fit()
reg_results.summary()

/home/yz685/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:130: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.671
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     43.23
Date:                Wed, 29 Sep 2021   Prob (F-statistic):               0.00
Time:                        10:42:30   Log-Likelihood:                -15362.
No. Observations:                2000   AIC:                         3.091e+04
Df Residuals:                    1909   BIC:                         3.142e+04
Df Model:                          90                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                      4539.6821   7172.662      0.633      0.527   -9527.396    1.86e+04
asymp_prob_mult                             293.5968   1141.565      0.257      0.797   -1945.250    2532.443
inital_prev_mult                          -1703.0148   1115.542     -1.527      0.127   -3890.824     484.794
R0                                         1681.3058    363.082      4.631      0.000     969.227    2393.384
outside_inf_mult                            805.2527   1087.826      0.740      0.459   -1328.200    2938.705
daily_self_report_prob                    -8025.5347   4020.767     -1.996      0.046   -1.59e+04    -139.977
ct_mult                                   -4568.1453   1149.485     -3.974      0.000   -6822.523   -2313.768
ct_testing_ratio                          -3458.8525   1105.289     -3.129      0.002   -5626.553   -1291.152
test_sensitivity                          -2438.0464   2884.661     -0.845      0.398   -8095.464    3219.371
test_noncompliance                         3569.4349    1.1e+04      0.326      0.745   -1.79e+04    2.51e+04
E_time                                    -1134.2855    559.904     -2.026      0.043   -2232.374     -36.197
ID_time                                     429.6112    576.888      0.745      0.457    -701.786    1561.009
Sy_time                                     236.0248   1034.924      0.228      0.820   -1793.676    2265.726
asymp_prob_mult^2                           388.5435    172.044      2.258      0.024      51.130     725.957
asymp_prob_mult inital_prev_mult            214.8467    151.927      1.414      0.157     -83.114     512.807
asymp_prob_mult R0                          742.5819     50.183     14.798      0.000     644.163     841.001
asymp_prob_mult outside_inf_mult             22.9639    149.602      0.153      0.878    -270.437     316.365
asymp_prob_mult daily_self_report_prob     -626.7457    548.055     -1.144      0.253   -1701.594     448.103
asymp_prob_mult ct_mult                   -1083.2437    149.092     -7.266      0.000   -1375.644    -790.843
asymp_prob_mult ct_testing_ratio          -1299.6514    150.813     -8.618      0.000   -1595.428   -1003.875
asymp_prob_mult test_sensitivity          -3316.4660    381.505     -8.693      0.000   -4064.677   -2568.255
asymp_prob_mult test_noncompliance         5539.3935   1524.442      3.634      0.000    2549.646    8529.141
asymp_prob_mult E_time                       49.9517     74.211      0.673      0.501     -95.591     195.494
asymp_prob_mult ID_time                     206.7098     76.522      2.701      0.007      56.634     356.786
asymp_prob_mult Sy_time                     104.1860     76.355      1.364      0.173     -45.562     253.934
ini

In [5]:
reg_results.params

const               4539.682082
asymp_prob_mult      293.596751
inital_prev_mult   -1703.014768
R0                  1681.305826
outside_inf_mult     805.252669
                       ...     
E_time ID_time       -38.637360
E_time Sy_time        55.321066
ID_time^2             18.595522
ID_time Sy_time        1.494890
Sy_time^2            -23.999865
Length: 91, dtype: float64

In [6]:
# get mean and standard deviation of parameters
param_names = reg_results.params.keys()[1:13]

mean_dict = dict()
sd_dict = dict()

for param in param_names:
    mean_dict[param] = (PARAM_BOUNDS[param][1] + PARAM_BOUNDS[param][0])/2
    sd_dict[param] = (PARAM_BOUNDS[param][1] - PARAM_BOUNDS[param][0])/(2*1.96)


mean_vals = np.array(list(mean_dict.values()))
sd_vals = np.array(list(sd_dict.values()))

In [7]:
mean_vals

array([ 1.  ,  1.  ,  2.5 ,  1.  ,  0.36,  1.5 ,  1.  ,  0.6 ,  0.1 ,
        2.  ,  3.  , 12.  ])

In [8]:
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint


In [9]:
# negative square root of the number of infections (taking the square root increases the optimizer's success rate)
def nonlinear_objective(x, reg_model = reg_model):
    assert len(x) == 12, 'x should be 12-dimensional'

    quadratic = PolynomialFeatures(degree = 2,interaction_only=False,include_bias = False)
    features = np.concatenate((np.array([1]),quadratic.fit_transform(x.reshape(1,-1))[0]))

    return -np.sqrt(max(reg_model.predict(reg_model.fit().params, features),0))

# ellipsoidal constraints
def ellipsoid_constraint_func_old(x, size=10, mean_vals = mean_vals, sd_vals = sd_vals): # what size to use? not 1.96^2
    assert len(x) == 12, 'x should be 12-dimensional'
    
    result = 0
    for i in range(12):
        result += (x[i] - mean_vals[i])**2 / (sd_vals[i]**2)
    result -= size
    return result

# ellipsoidal constraints
def ellipsoid_constraint_func_new(x, size=21.03, mean_vals = mean_vals, sd_vals = sd_vals): # what size to use? not 1.96^2
    assert len(x) == 12, 'x should be 12-dimensional'
    
    result = 0
    for i in range(12):
        result += (x[i] - mean_vals[i])**2 / (sd_vals[i]**2)
    result -= size
    return result


ellip_cons_old = NonlinearConstraint(ellipsoid_constraint_func_old, -np.inf, 0)
ellip_cons_new = NonlinearConstraint(ellipsoid_constraint_func_new, -np.inf, 0)


# test the method on a simple objective function
def test_objective(x, mean_vals = mean_vals):
    return sum((x - mean_vals)**2)

In [10]:
# test on simple objective function to verify it works

count_success = 0

for _ in range(100):

    initial_point = np.random.rand(12)

    result = minimize(test_objective, initial_point, 
             constraints = ellip_cons,
             method = 'SLSQP')


    if result.success:
        count_success += 1


print(count_success)
print(result)

NameError: name 'ellip_cons' is not defined

In [37]:
# Now compute optimizer of quadratic regression objective

#count_success = 0
solutions = []

for _ in range(1000):

    #initial_point = np.random.rand(12) * mean_vals*2
    
    # sample from prior mean +- Unif(0,3) * prior SD
    initial_point = mean_vals + 6*(np.random.rand(12) - np.ones(12)*1/2)*sd_vals

    result = minimize(nonlinear_objective, initial_point, 
             constraints = ellip_cons,
             method = 'SLSQP')
    
    solutions.append(result.x)

    #if result.success:
    #    count_success += 1

#print(count_success)

In [11]:
# Now compute optimizer of quadratic regression objective

#count_success = 0
solutions_new = []

for _ in range(1000):

    #initial_point = np.random.rand(12) * mean_vals*2
    
    # sample from prior mean +- Unif(0,3) * prior SD
    initial_point = mean_vals + 6*(np.random.rand(12) - np.ones(12)*1/2)*sd_vals

    result = minimize(nonlinear_objective, initial_point, 
             constraints = ellip_cons_new,
             method = 'SLSQP')
    
    solutions_new.append(result.x)

    #if result.success:
    #    count_success += 1

#print(count_success)

In [12]:
# function for clustering optimizer outputs 
def cluster_data(data, threshold):
    
    solutions = [[]]
    
    for i in range(len(data)):       
        if solutions[0] == []:
            solutions[0].append(data[i])
        else:
            is_new_sol = True
            for j in range(len(solutions)):
                
                if len(solutions[j]) == 1:
                    current_cluster_mean = solutions[j][0]
                else:
                    current_cluster_mean = np.mean(np.stack(solutions[j]), axis=0)
                
                if np.linalg.norm(data[i]-current_cluster_mean)< threshold:
                    is_new_sol = False
                    solutions[j].append(data[i])
                    break
            
            if is_new_sol:
                solutions.append([data[i]])
            
    solutions.sort(key=len, reverse=True)
    
    return solutions

In [13]:
clustered_solutions_new = cluster_data(solutions_new, 0.001)


In [14]:
cluster_means_new = []

for j in range(len(clustered_solutions_new)):
    cluster_mean = np.mean(np.stack(clustered_solutions_new[j]), axis=0)
    cluster_means_new.append(cluster_mean)
    print('Solution {} ({}/1000): {} \n'.format(j, len(clustered_solutions_new[j]), cluster_mean))

Solution 0 (584/1000): [ 1.33471705  1.05207254  4.80206845  1.01659074  0.32791749  1.04367832
  0.555386    0.41700678  0.11236459  1.97460645  3.25227984 12.01972775] 

Solution 1 (319/1000): [ 0.67322786  0.96304501  0.24263633  0.99901463  0.39488123  1.97771284
  1.4584699   0.77980477  0.08709444  2.02521236  2.74178755 11.9825903 ] 

Solution 2 (1/1000): [ 0.62243349  0.74840147  4.0043835   0.62855057  0.44617536  1.66680631
  1.33955882  0.59711187  0.10632496  1.07271135  3.7701708  12.80461863] 

Solution 3 (1/1000): [ 0.56498228  0.85596383  0.40938449  1.28097639  0.3308358   1.1024126
  0.98550573  0.43915794  0.09113666  2.42084544  2.26338332 12.51799735] 

Solution 4 (1/1000): [ 1.35894656  1.05529736  1.18603959  1.20309964  0.48647014  1.50640847
  0.92095531  0.45749983  0.05558875  3.02914776  3.55328979 11.30099118] 

Solution 5 (1/1000): [ 1.45236346  1.24804253  1.43267273  1.41309238  0.30309587  1.06864934
  0.88659305  0.79643184  0.11455998  1.05447789  2.8

In [42]:
quadratic = PolynomialFeatures(degree = 2,interaction_only=False,include_bias = False)

sol_0 = cluster_means[0]
features_0 = np.concatenate((np.array([1]),quadratic.fit_transform(sol_0.reshape(1,-1))[0]))

sol_1 = cluster_means[1]
features_1 = np.concatenate((np.array([1]),quadratic.fit_transform(sol_1.reshape(1,-1))[0]))

obj_0 = reg_model.predict(reg_model.fit().params, features_0)
obj_1 = reg_model.predict(reg_model.fit().params, features_1)

print('Objective value for solution 0 and 1: {}, {}'.format(obj_0, obj_1))


Objective value for solution 0 and 1: 4872.193044095273, 2102.5697626011347


In [15]:
quadratic = PolynomialFeatures(degree = 2,interaction_only=False,include_bias = False)

sol_0 = cluster_means_new[0]
features_0 = np.concatenate((np.array([1]),quadratic.fit_transform(sol_0.reshape(1,-1))[0]))

sol_1 = cluster_means_new[1]
features_1 = np.concatenate((np.array([1]),quadratic.fit_transform(sol_1.reshape(1,-1))[0]))

obj_0 = reg_model.predict(reg_model.fit().params, features_0)
obj_1 = reg_model.predict(reg_model.fit().params, features_1)

print('Objective value for solution 0 and 1: {}, {}'.format(obj_0, obj_1))


Objective value for solution 0 and 1: 9437.80728508142, 5420.707795344806


In [16]:
# This means solution 0 is what we want

nonlin_pess = dict(zip(param_names, cluster_means_new[0]))
nonlin_pess

{'asymp_prob_mult': 1.3347170451588275,
 'inital_prev_mult': 1.052072539291337,
 'R0': 4.8020684479117035,
 'outside_inf_mult': 1.0165907376762244,
 'daily_self_report_prob': 0.3279174908206987,
 'ct_mult': 1.0436783232271167,
 'ct_testing_ratio': 0.5553859986215729,
 'test_sensitivity': 0.41700678018840226,
 'test_noncompliance': 0.11236458819144107,
 'E_time': 1.9746064475997762,
 'ID_time': 3.252279835676852,
 'Sy_time': 12.019727750861811}

In [17]:
# juxtapose it with pessimistic from linear model

df = pd.read_csv('./pessimistic_point_linear_model.csv')
df = df.rename(columns={"Unnamed: 0": "Parameter"})

In [18]:
nonlin_pess_col = []

for param_name in list(df['Parameter']):
    #print(nonlin_pess[param_name])
    nonlin_pess_col.append(nonlin_pess[param_name])
    
df['new pess from quadratic model'] = nonlin_pess_col

In [19]:
df

,Parameter,pess from linear model,new pess from quadratic model
0,R0,3.360705,4.802068
1,E_time,1.981626,1.974606
2,ID_time,3.084056,3.252280
3,ct_testing_ratio,0.866472,0.555386
4,ct_mult,1.345780,1.043678
5,daily_self_report_prob,0.348582,0.327917
6,test_noncompliance,0.104338,0.112365
7,outside_inf_mult,1.020257,1.016591
8,test_sensitivity,0.533736,0.417007
9,inital_prev_mult,1.043116,1.052073


In [21]:
df.to_csv('./pessimistic_linear_quadratic_comparison.csv')